To be honest, I don't think this code is working really well, because the simulations look very weird... (you can find them in the folder gifs_lucia)

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import imageio

In [2]:
it = 0

In [69]:
it += 1
N = 50
timesteps = 500 # 50 seconds
boundaries = 100

#parameters are: zor, zoo, zoa, alpha, theta, s, sigma, tau
params = [5, 10, 15, 300, 50, 3, 5, 0.1]
params_string = f'it={it}, zor={params[0]}, zoo={params[1]}, zoa={params[2]}, alpha={params[3]}, theta={params[4]}, s={params[5]}, sigma={params[6]}, tau={params[7]}'

init = np.random.rand(N, 4)

In [70]:
def r_ij(arr, i, j):
    '''returns unit vector from individual i in direction of individual j'''
    diff = arr[j, :2]-arr[i, :2]
    return diff/np.linalg.norm(diff)

def ZOR(arr, i, params, boundaries, check_speed=True):
    if check_speed:
        s = params[5]
        tau = params[7]
        r_r = params[0]
        assert s*tau<r_r
    l = []
    for j in range(N):
        if j != i:
            diff = arr[j, :2]-arr[i, :2]
            if boundaries[0]<=np.linalg.norm(diff)<boundaries[1]: # make sure that fish is inside zor
                R = r_ij(arr, i, j)/np.linalg.norm(r_ij(arr, i, j))
                l.append(R)
    return -sum(l), len(l)
    
def ZOO(arr, i, params):
    r_r = params[0]
    r_o = params[1]
    l = []
    for j in range(N):
        diff = arr[j, -2:]-arr[i, -2:]
        if r_r<=np.linalg.norm(diff)<r_o: # make sure that fish is inside zoo
            R = arr[j, -2:]/np.linalg.norm(arr[j, -2:])
            l.append(R)
    return sum(l), len(l)

def ZOA(arr, i, params, boundaries, check_speed=False):
    return -ZOR(arr, i, params, boundaries)[0], ZOR(arr, i, params, boundaries)[1]
    

In [71]:
arr = init
arrays = []
for timestep in range(timesteps):
    arr_new = arr.copy()
    for i in range(N):
        zor, n_r = ZOR(arr, i, params, (0, params[0]))
        zoo, n_o = ZOO(arr, i, params)
        zoa, n_a = ZOA(arr, i, params, (params[1], params[2]))

        if n_r>0:
            vel = zor

        elif n_o>0:
            if n_a==0:
                vel = zoo
            else:
                vel = (1/2)*(zoo+zoa)

        elif n_a>0:
            if n_o==0:
                vel = zoa
            else:
                vel = (1/2)*(zoo+zoa)

        if vel.all()==0:
            vel = arr[i, -2:]
        
        # periodic boundary conditions 
        if abs(arr[i, 0]) >= boundaries or abs(arr[i, 1]) >= boundaries:
            arr[i, 0] = -arr[i, 0]
            arr[i, 1] = -arr[i, 1]

        arr_new[i] = [arr[i, 0]+arr[i, 2], arr[i, 1]+arr[i, 3], vel[0], vel[1]]
    
    arrays.append(arr_new)
    arr = arr_new
    

In [72]:
arrays[2]

array([[-5.24976701e+01, -4.24413587e+01, -8.78539096e-01,
        -4.77670448e-01],
       [ 4.88920926e+01, -4.53465457e+01,  6.89628975e-01,
         7.24162880e-01],
       [-1.11942143e+01,  2.75005368e+01,  1.69680325e+00,
        -3.76367028e-01],
       [-3.92511710e+00, -7.21339372e+01, -5.64003534e-01,
        -4.46899721e-01],
       [-5.36708173e+00, -1.44352876e+01, -1.84314306e-01,
        -9.82867354e-01],
       [ 4.50860592e+01, -7.17669262e+01,  9.22007270e-01,
        -3.87172563e-01],
       [ 3.44808121e+01,  5.59919988e+01,  1.94199579e-01,
        -9.80962040e-01],
       [ 4.76023090e+01, -2.44756875e+01, -3.76882970e-01,
        -8.40737708e-01],
       [-3.44453796e+01, -2.76475952e+01, -7.11304679e-01,
        -7.02883812e-01],
       [ 2.95482539e+01,  2.44622158e+01,  8.80909649e-01,
         4.73284471e-01],
       [-3.50218601e+01, -9.46396006e+00, -1.77277036e+00,
        -1.12385903e+00],
       [-7.48410289e+00, -1.10017222e+01, -6.64861389e-01,
      

In [73]:
png_list = []
for i in range(timesteps):
    x = arrays[i][:, 0] 
    y = arrays[i][:, 1]
    u = arrays[i][:, 2]
    v = arrays[i][:, 3]
    fig = plt.figure(figsize=(10, 10))
    plt.quiver(x, y, u, v) # https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.quiver.html
    plt.xlim(-boundaries, boundaries)
    plt.ylim(-boundaries, boundaries)
    plt.savefig(f'plots_lucia/{i}.png')
    png_list.append(f'plots_lucia/{i}.png')
    plt.close()


In [74]:
with imageio.get_writer('gifs_lucia/'+params_string+'.gif', mode='I') as writer:
    for filename in png_list:
        image = imageio.imread(filename)
        writer.append_data(image)